### Import necessary libraries

In [ ]:
import pandas as pd
import re
import plotly.express as px
import json
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import pearsonr
import seaborn as sns
import pickle
from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
# Load the variable from the file
with open('path_variable.pkl', 'rb') as f:
    path_variable = pickle.load(f)

# if this doesn't work just manually give the path the same value as you gave at retrieving_data.ipynb file
# change the PATH to the location of your folder where you store the geojson file
# PATH = '/Users/ansat.omurzakov/Desktop/TUe/Data Challenge 2/data/'
PATH = path_variable.as_posix()

### Read the met street crimes csv file

In [ ]:
data1 = pd.read_csv(PATH + '/metropolitan-street.csv')

### Retrieve boroughs from LSOA names to be able to generalize findings in the future
- put in appropriate timeframe when pas data contains values (2016-2023)
- get month and year columns
- get rid of rows where you don't have LSOA names
- get rid of rows where you don't have long, lat

In [ ]:
def get_borough(value):
    borough = re.split(r'\d+[A-Z]', value)[0].strip()
    return borough

#### Some data cleaning

In [ ]:
# extract year and month
data1['year'] = data1['Month'].apply(lambda x: int(x.split('-')[0]))
data1['month'] = data1['Month'].apply(lambda x: int(x.split('-')[1]))

# get data in appropriate time span
data1 = data1[data1['year'].between(2016,2023)]

# leave out the data which doesn't contain LSOA names or Longitudes or Latitudes
data1 = data1[data1['LSOA name'].isna() == False]
data1 = data1[(data1['Longitude'].isna()== False) | (data1['Latitude'].isna() == False)]

# Retrieve boroughs from the LSOA name
data1['borough'] = data1['LSOA name'].apply(get_borough)

# Drop duplicates
data1.drop_duplicates(keep = 'first', inplace = True)

### Checking if boroughs are controlled by the MOPAC

In [ ]:
with open(PATH + '/boroughs_neighbourhoods.json', 'r') as file:
    json_data = json.load(file)

# Convert JSON data to a DataFrame
boroughs_data = []
for borough, area_codes in json_data.items():
    for area_code in area_codes:
        boroughs_data.append({'borough': borough, 'neighbourhood': area_code})

boroughs_neighbours = pd.DataFrame(boroughs_data)

In [ ]:
data1 = data1[data1['borough'].isin(boroughs_neighbours['borough'].unique())]

In [ ]:
# Drop irrelevant columns from the dataset
data1 = data1.drop(['Crime ID', 'Reported by', 'Falls within', 'LSOA code', 'LSOA name', 'Context', 'Month'], axis = 1).reset_index(drop = True)

### Cleaning locations of crimes and finding the meaning to generalize them to make some statistical analysis

In [ ]:
data1['cleaned_location'] = data1['Location'].apply(lambda x: x.lower().split('on or near')[1].strip())

In [ ]:
def categorize_location_v4(location):
    
    residential_keywords = [
        'cottages', 'house', 'close', 'mews', 'court', 
        'place', 'building', 'village', 'villa', 
        'estate', 'palace', 'bldgs', 'yard', 'crescent',
        'sanctuary', 'colonade', 'terrace',
    ] # no change needed

    commercial_keywords = [
        'wharf', 'pier', 'quay', 'arcade', 'mall', 'plaza', 
        'market', 'square', 'parade', 'shopping', 'alley', 
        'post office', 'bank', 'circus', 'nightclub', 'golf', 
        'boulevard',
    ] # no change needed


    public_amenities_keywords = [
        'hospital', 'school', 'college', 'academy',  
        'park', 'education', 'church', 'sports', 
        'theatre',  'towers', 'assembly', 'police station', 
        'chantry', 'centre'
    ]

    road_pathways_keywords = list(
        set(['track', 'bridge', 'circle', 'ring', 
            'underpass', 'tunnel', 'flyover', 'route', 
            'road', 'street', 'drive', 'path', 'link', 
            'gate', 'passage', 'slope', 'corner', 'end', 
            'mount', 'ride', 'rise', 'walk', 'way', 'quadrant', 
            'lane', 'row', 'arch', 'ring', 'avenue', 'road', 
            'street', 'knightsbridge', 'gate', 'passage',
            'slope', 'corner', 'end','mount','ride','rise', 
            'walk',  'way', 'quadrant', 'lane','row', 'tunnel', 
            'riverside', 'approach', 'pavement','collonade', 
            'crossing', 'cross'])
    )

    transportation_keywords = [
        'bus', 'tram', 'train', 'ferry', 'dock', 'terminal', 'aerodrome', 
        'airport', 'subway', 'bus station', 'petrol station', 'gas station', 
        'station',
    ]

    natural_features_keywords = [
        'glade', 'dale', 'ridge', 'wood', 'forest', 'mead', 'tree', 'ground', 
        'foreshore', 'canal', 'peak', 'lake', 'orchard', 'coppice',  'wheatlands', 
        'pond','garden', 'knoll', 'copse', 'vale','field', 'meadow','hill',
        'grove', 'green', 'lawns', 'commons', 'heath', 'mills'
    ]


    location_lower = location
    
    if any(keyword in location_lower for keyword in residential_keywords):
        return 'Residential Area'
    elif any(keyword in location_lower for keyword in commercial_keywords):
        return 'Commercial Area'
    elif any(keyword in location_lower for keyword in public_amenities_keywords):
        return 'Public Amenities'
    elif any(keyword in location_lower for keyword in natural_features_keywords):
        return 'Natural Features'
    elif any(keyword in location_lower for keyword in transportation_keywords):
        return 'Transportation'
    elif any(keyword in location_lower for keyword in road_pathways_keywords):
        return 'Road_Pathway'
    elif re.search(r'\w\d+', location_lower):
        return 'Road_Pathway'
    else:
        return 'Other'

# Apply the enhanced categorization to the dataframe
data1['category'] = data1['cleaned_location'].apply(categorize_location_v4)

# Display the updated summary of categorized data
further_enhanced_category_counts = data1['category'].value_counts()
print(further_enhanced_category_counts)

In [ ]:
fig = px.histogram(data1['category'])

fig.update_layout(
    title="Distribution of data per locations (in London)",
    xaxis_title="Location category",
    yaxis_title="# of crimes occuring")

fig.show();

### Finding the distribution of Last outcome percentage

In [ ]:
cat = data1.groupby('Last outcome category').count().sort_values(by = 'category', ascending = False)
cat['percentage'] = cat['category']/sum(cat['category']) * 100

In [ ]:
cat[['percentage']]

### Grouping crimes in categories just as with places

In [ ]:
def give_sev_idx(crime):
    # (5)
    if crime in ['Violence and sexual offences', 'Criminal damage and arson']:
        return 'Violent crime'

    # (4)
    elif crime in ['Burglary', 'Vehicle crime', 'Robbery', 'Theft from the person',  'Shoplifting', 'Bicycle theft']:
        return 'Property damage'
    
    # (3) - 
    elif crime in [ 'Public order', 'Anti-social behaviour',  'Other crime']:
        return 'Public order crime'

    # (2)
    elif crime in ['Drugs', 'Possession of weapons']:
        return 'Drugs and weapons'
    
    # (1)
    elif crime in ['Other crime', 'Other theft']:
        return 'Miscellaneous crime'

### Getting quarter from the data since the PAS data is given with years and quarters

In [ ]:
def give_quarter(month):
    if month in [1,2,3]:
        return 1
    elif month in [4,5,6]:
        return 2
    elif month in [7,8,9]:
        return 3
    elif month in [10,11,12]:
        return 4

In [ ]:
data1['crime_group'] = data1['Crime type'].apply(give_sev_idx)
data1['quarter'] = data1['month'].apply(give_quarter)

In [ ]:
data1.head()

### Creating a street dataframe for merging with file we created at data_analysis.ipynb

In [ ]:
data_for_merge = data1[['Crime type', 'Last outcome category', 'borough', 'quarter', 'year', 'category', 'crime_group']]

In [ ]:
columns_to_count = ['Crime type', 'Last outcome category', 'category', 'crime_group']
dummies = pd.get_dummies(data_for_merge[columns_to_count])
street_cleaned_with_dummies = pd.concat([data_for_merge, dummies], axis=1)

In [ ]:
wrap = street_cleaned_with_dummies.drop(['Crime type', 'Last outcome category', 'category', 'crime_group'], axis = 1)

In [ ]:
wrap = wrap.groupby(['borough', 'year', 'quarter']).sum().reset_index()
wrap

In [ ]:
pas_with_sas = pd.read_csv(PATH + '/PAS_with_SAS.csv')
pas_with_sas = pas_with_sas.drop('Unnamed: 0', axis = 1)

### Saving the file, so you could delete the metropolitan-street.csv which weights around 3 gb, creating a smaller in volumne file for your RAM to be easier to handle

In [ ]:
pas_with_sas.merge(wrap, on = ['borough', 'year', 'quarter']).to_csv(PATH + '/final_latest_pas_with_sas_with_street_new.csv')

In [ ]:

pss = pd.read_csv(PATH + '/FINAL_agg_Dataset.csv')
pss.drop('Unnamed: 0', axis = 1, inplace = True)

In [ ]:
pss.head()

# Graphics indicating at which locations crimes mainly happening per borough

### Saving columns for places categories

In [ ]:
pipap = []
for column in pss.columns:
    if column.startswith('category'):
        pipap.append(column)

## Unscaled graphic

In [ ]:
siki = pss[pipap + ['year', 'quarter', 'borough']]

# Create 'Year-Quarter' column
siki['Year-Quarter'] = siki['year'].astype(str) + ' Q' + siki['quarter'].astype(str)

# Define categories
categories = [
    'category_Commercial Area', 
    'category_Natural Features',
    'category_Other', 
    'category_Public Amenities',
    'category_Residential Area', 
    'category_Transportation',
    'category_Road_Pathway'
]

# Get the unique boroughs
boroughs = siki['borough'].unique()

# Create a consistent color palette
colors = px.colors.qualitative.Plotly
color_mapping = {category: colors[i % len(colors)] for i, category in enumerate(categories)}

# Create subplot figure with shared x and y axes
fig = make_subplots(rows=4, cols=8, subplot_titles=boroughs)

# Iterate over each borough and add a subplot
for i, borough in enumerate(boroughs):
    # Group by 'Year-Quarter' and sum the counts for the current borough
    df_grouped = siki[siki['borough'] == borough].groupby('Year-Quarter')[categories].sum().reset_index()
    
    # Unpivot the grouped DataFrame to long format
    df_long = df_grouped.melt(id_vars=['Year-Quarter'], 
                              value_vars=categories,
                              var_name='Category', 
                              value_name='Count')
    
    # Rename categories to remove 'category_' prefix for cleaner labels
    df_long['Category'] = df_long['Category'].str.replace('category_', '')
    
    # Determine subplot row and column
    row = i // 8 + 1
    col = i % 8 + 1
    
    # Add traces for each category with consistent colors
    for category in df_long['Category'].unique():
        category_data = df_long[df_long['Category'] == category]
        fig.add_trace(
            go.Scatter(x=category_data['Year-Quarter'], y=category_data['Count'], 
                       mode='lines+markers', name=category, legendgroup=category,
                       marker=dict(color=color_mapping['category_' + category]), # Use consistent color
                       showlegend=(i == 0)),
            row=row, col=col
        )
    
    # Update subplot title
    fig.update_xaxes(title_text="Year-Quarter", row=row, col=col)
    fig.update_yaxes(title_text="Count", row=row, col=col)

# Update layout
fig.update_layout(
    height=1500, width=3000,
    title_text="Crime Count per Category by Year and Quarter for All Boroughs",
    showlegend=True,
    title_font_size=30,
    title_font_family='bold'
)

# Show the figure
fig.show()


## Scaled similarly

In [ ]:
# Create 'Year-Quarter' column
siki['Year-Quarter'] = siki['year'].astype(str) + ' Q' + siki['quarter'].astype(str)

# Define categories
categories = [
    'category_Commercial Area', 
    'category_Natural Features',
    'category_Other', 
    'category_Public Amenities',
    'category_Residential Area', 
    'category_Transportation',
    'category_Road_Pathway'
]

# Get the unique boroughs
boroughs = siki['borough'].unique()

# Create a consistent color palette
colors = px.colors.qualitative.Plotly
color_mapping = {category: colors[i % len(colors)] for i, category in enumerate(categories)}

# Determine the overall x and y axis ranges
overall_min_x = siki['Year-Quarter'].min()
overall_max_x = siki['Year-Quarter'].max()
overall_min_y = min(siki[['category_Commercial Area', 
    'category_Natural Features',
    'category_Other', 
    'category_Public Amenities',
    'category_Residential Area', 
    'category_Transportation',
    'category_Road_Pathway']].min().to_list())
overall_max_y = max(siki[[ 'category_Commercial Area', 
    'category_Natural Features',
    'category_Other', 
    'category_Public Amenities',
    'category_Residential Area', 
    'category_Transportation',
    'category_Road_Pathway']].max().to_list())

# Create subplot figure with shared x and y axes
fig = make_subplots(rows=4, cols=8, subplot_titles=boroughs)

# Iterate over each borough and add a subplot
for i, borough in enumerate(boroughs):
    # Group by 'Year-Quarter' and sum the counts for the current borough
    df_grouped = siki[siki['borough'] == borough].groupby('Year-Quarter')[categories].sum().reset_index()
    
    # Unpivot the grouped DataFrame to long format
    df_long = df_grouped.melt(id_vars=['Year-Quarter'], 
                              value_vars=categories,
                              var_name='Category', 
                              value_name='Count')
    
    # Rename categories to remove 'category_' prefix for cleaner labels
    df_long['Category'] = df_long['Category'].str.replace('category_', '')
    
    # Determine subplot row and column
    row = i // 8 + 1
    col = i % 8 + 1
    
    # Add traces for each category with consistent colors
    for category in df_long['Category'].unique():
        category_data = df_long[df_long['Category'] == category]
        fig.add_trace(
            go.Scatter(x=category_data['Year-Quarter'], y=category_data['Count'], 
                       mode='lines+markers', name=category, legendgroup=category,
                       marker=dict(color=color_mapping['category_' + category]), # Use consistent color
                       showlegend=(i == 0)),
            row=row, col=col
        )
    
    # Update subplot title
    fig.update_xaxes(title_text="Year-Quarter", range=[overall_min_x, overall_max_x], row=row, col=col)
    fig.update_yaxes(title_text="Count", range=[overall_min_y, overall_max_y], row=row, col=col)

# Update layout
fig.update_layout(
    height=1500, width=2500,
    title_text="Crime Count per Location Category by Year and Quarter for All Boroughs",
    showlegend=True,
    title_font_size=30,
    title_font_family='bold'
)

# Show the figure
fig.show()

# Graphics indicating at what kind of crimes mainly happening per borough

## Saving crime groups in a list

In [ ]:
crime_group = []
for column in pss.columns:
    if column.startswith('crime_group'):
        crime_group.append(column)

## Unscaled grpahic

In [ ]:
siki = pss[crime_group + ['year', 'quarter', 'borough']]

# Create 'Year-Quarter' column
siki['Year-Quarter'] = siki['year'].astype(str) + ' Q' + siki['quarter'].astype(str)

# Define categories
categories = [
    'crime_group_Drugs and weapons',
    'crime_group_Miscellaneous crime',
    'crime_group_Property damage',
    'crime_group_Public order crime',
    'crime_group_Violent crime'
]

# Get the unique boroughs
boroughs = siki['borough'].unique()

# Create a consistent color palette
colors = px.colors.qualitative.Plotly
color_mapping = {category: colors[i % len(colors)] for i, category in enumerate(categories)}

# Create subplot figure with shared x and y axes
fig = make_subplots(rows=4, cols=8, subplot_titles=boroughs)

# Iterate over each borough and add a subplot
for i, borough in enumerate(boroughs):
    # Group by 'Year-Quarter' and sum the counts for the current borough
    df_grouped = siki[siki['borough'] == borough].groupby('Year-Quarter')[categories].sum().reset_index()
    
    # Unpivot the grouped DataFrame to long format
    df_long = df_grouped.melt(id_vars=['Year-Quarter'], 
                              value_vars=categories,
                              var_name='Category', 
                              value_name='Count')
    
    # Rename categories to remove 'category_' prefix for cleaner labels
    df_long['Category'] = df_long['Category'].str.replace('crime_', '')
    
    # Determine subplot row and column
    row = i // 8 + 1
    col = i % 8 + 1
    
    # Add traces for each category with consistent colors
    for category in df_long['Category'].unique():
        category_data = df_long[df_long['Category'] == category]
        fig.add_trace(
            go.Scatter(x=category_data['Year-Quarter'], y=category_data['Count'], 
                       mode='lines+markers', name=category, legendgroup=category,
                       marker=dict(color=color_mapping['crime_' + category]), # Use consistent color
                       showlegend=(i == 0)),
            row=row, col=col
        )
    
    # Update subplot title
    # fig.update_xaxes(title_text="Year-Quarter", range=[overall_min_x, overall_max_x], row=row, col=col)
    # fig.update_yaxes(title_text="Count", range=[overall_min_y, overall_max_y], row=row, col=col)

# Update layout
fig.update_layout(
    height=1500, width=3000,
    title_text="Crime Count per Category by Year and Quarter for All Boroughs",
    showlegend=True,
    title_font_size=30,
    title_font_family='bold'
)

# Show the figure
fig.show()

## Scaled similarly 

In [ ]:
# Create 'Year-Quarter' column
siki['Year-Quarter'] = siki['year'].astype(str) + ' Q' + siki['quarter'].astype(str)

# Define categories
categories = [
    'crime_group_Drugs and weapons',
    'crime_group_Miscellaneous crime',
    'crime_group_Property damage',
    'crime_group_Public order crime',
    'crime_group_Violent crime'
]

# Get the unique boroughs
boroughs = siki['borough'].unique()

# Create a consistent color palette
colors = px.colors.qualitative.Plotly
color_mapping = {category: colors[i % len(colors)] for i, category in enumerate(categories)}

overall_min_x = siki['Year-Quarter'].min()
overall_max_x = siki['Year-Quarter'].max()
overall_min_y = min(siki[['crime_group_Drugs and weapons',
    'crime_group_Miscellaneous crime',
    'crime_group_Property damage',
    'crime_group_Public order crime',
    'crime_group_Violent crime']].min().to_list())
overall_max_y = max(siki[['crime_group_Drugs and weapons',
    'crime_group_Miscellaneous crime',
    'crime_group_Property damage',
    'crime_group_Public order crime',
    'crime_group_Violent crime']].max().to_list())

# Determine the overall x and y axis ranges
overall_min_x = siki['Year-Quarter'].min()
overall_max_x = siki['Year-Quarter'].max()
overall_min_y = min(siki[crime_group].min().to_list())
overall_max_y = max(siki[crime_group].max().to_list())

# Create subplot figure with shared x and y axes
fig = make_subplots(rows=4, cols=8, subplot_titles=boroughs)

# Iterate over each borough and add a subplot
for i, borough in enumerate(boroughs):
    # Group by 'Year-Quarter' and sum the counts for the current borough
    df_grouped = siki[siki['borough'] == borough].groupby('Year-Quarter')[categories].sum().reset_index()
    
    # Unpivot the grouped DataFrame to long format
    df_long = df_grouped.melt(id_vars=['Year-Quarter'], 
                              value_vars=categories,
                              var_name='Category', 
                              value_name='Count')
    
    # Rename categories to remove 'category_' prefix for cleaner labels
    df_long['Category'] = df_long['Category'].str.replace('crime_', '')
    
    # Determine subplot row and column
    row = i // 8 + 1
    col = i % 8 + 1
    
    # Add traces for each category with consistent colors
    for category in df_long['Category'].unique():
        category_data = df_long[df_long['Category'] == category]
        fig.add_trace(
            go.Scatter(x=category_data['Year-Quarter'], y=category_data['Count'], 
                       mode='lines+markers', name=category, legendgroup=category,
                       marker=dict(color=color_mapping['crime_' + category]), # Use consistent color
                       showlegend=(i == 0)),
            row=row, col=col
        )
    
    # Update subplot title
    # fig.update_xaxes(title_text="Year-Quarter", row=row, col=col)
    # fig.update_yaxes(title_text="Count", row=row, col=col)
    fig.update_xaxes(title_text="Year-Quarter", range=[overall_min_x, overall_max_x], row=row, col=col)
    fig.update_yaxes(title_text="Count", range=[overall_min_y, overall_max_y], row=row, col=col)

# Update layout
fig.update_layout(
    height=1500, width=3000,
    title_text="Crime Count per Crime Category by Year and Quarter for All Boroughs",
    showlegend=True,
    title_font_size=30,
    title_font_family='bold'
)

# Show the figure
fig.show()


# Creating corrleation heatmap throughout 2016-2023 time period, between the most significant features at crime groups and crime locations

In [ ]:
to_get = pss[pipap+crime_group + ['year']]

In [ ]:
pipap = ['category_Natural Features',
 'category_Residential Area',
 'category_Road_Pathway',]

In [ ]:
crime_group = ['crime_group_Property damage','crime_group_Public order crime','crime_group_Violent crime']

In [ ]:
# Create subplots
fig, axes = plt.subplots(2, 4, figsize=(24, 12))
axes = axes.flatten()

# Initialize a dictionary to store correlation matrices
correlation_matrices = {}

# Loop over each year and calculate the correlation matrix
for i, year in enumerate(to_get['year'].unique()):
    year_data = to_get[to_get['year'] == year]  # Ensure year_data is a DataFrame
    correlation_matrix = year_data[pipap + crime_group].corr()
    correlation_matrices[year] = correlation_matrix

    # Extract the part of the correlation matrix that is relevant
    correlation_submatrix = correlation_matrix.loc[pipap, crime_group]

    # Plot the heatmap for the current year
    sns.heatmap(correlation_submatrix, annot=True, fmt='.2f', cmap='magma', cbar=True, ax=axes[i])
    axes[i].set_title(f"Correlation Matrix for {year}")
    # Rotate x-axis tick labels and align them properly
    plt.setp(axes[i].get_xticklabels(), rotation=45, ha='right')

    # Ensure y-axis tick labels remain horizontal
    plt.setp(axes[i].get_yticklabels(), rotation=0, ha='right')
    # axes[i].tick_params(axis='y', rotation=0)

# Adjust layout
fig.suptitle('Correlation matrices for Crime groups and Places', weight = 'bold', size = 16)
plt.tight_layout()
plt.show();

### Checking the significance level as well as the correlation coefficient between different features and Confidence of the Trust

In [ ]:
# Generate sample data (replace with your actual data)
# Assume trust and crime_type are pandas Series with datetime index
trust = pss[(pss['year'].between(2017, 2023)) &(pss['borough'] == 'Richmond upon Thames')]['Contact ward officer']
crime_type = pss[(pss['year'].between(2017, 2023))& (pss['borough'] == 'Richmond upon Thames')]['"Good Job" local']

# Define window size for rolling correlation
window_size = 3  # 12 months per year

# Calculate rolling correlation
rolling_correlation = trust.rolling(window=window_size).corr(crime_type)

# Calculate statistical significance (p-values)
rolling_p_values = trust.rolling(window=window_size).apply(lambda x: pearsonr(x, crime_type.iloc[-len(x):])[1], raw=True)

# Plot rolling correlation
plt.figure(figsize=(10, 6))
rolling_correlation.plot(label='Rolling Correlation (Trust vs Crime Type)')
plt.axhline(y=0, color='gray', linestyle='--')
plt.title('Rolling Correlation between Trust and Crime Type')
plt.xlabel('Time')
plt.ylabel('Correlation Coefficient')
plt.legend()
plt.grid(True)

# Plot statistical significance
plt.figure(figsize=(10, 6))
rolling_p_values.plot(label='Rolling P-Values (Statistical Significance)')
plt.axhline(y=0.05, color='r', linestyle='--', label='Significance Threshold (p=0.05)')
plt.title('Statistical Significance of Rolling Correlation')
plt.xlabel('Time')
plt.ylabel('P-Value')
plt.legend()
plt.grid(True)

plt.show();

In [ ]:
fun = pd.concat([rolling_p_values, rolling_correlation], axis = 1)
fun = fun.rename(columns = {'Contact ward officer': 'p_value', 0: 'correlation value'})
fun[fun['p_value'] <= 0.05].median()